# **Cuaderno de ETL: De Datos Abiertos a un Modelo de Estrella**

**Objetivo:** Tomar un conjunto de datos públicos sobre estadísticas de educación en Colombia y transformarlo en un modelo dimensional de estrella, listo para ser analizado con herramientas de Business Intelligence.

**Fuente de Datos:** [MEN_ESTADISTICAS_EN_EDUCACION_EN_PREESCOLAR-B-SICA](https://www.datos.gov.co/Educaci-n/MEN_ESTADISTICAS_EN_EDUCACION_EN_PREESCOLAR-B-SICA/nudc-7mev/about_data)

**Nuestro Modelo de Estrella a Construir:**

* **Tabla de Hechos (Fact_Matriculas):**
    * `id_tiempo` (FK)
    * `id_geografia` (FK)
    * `total_matriculados` (Métrica)
    ...

* **Tablas de Dimensiones:**
    * `Dim_Tiempo` (año)
    * `Dim_Geografia` (departamento, municipio)

¡Manos a la obra!

In [208]:
# ===================================================================
# PASO 1: CONFIGURACIÓN E INSTALACIÓN DE LIBRERÍAS
# ===================================================================

import pandas as pd
import requests
import sqlite3

print("✅ Librerías importadas.")

# ===================================================================
# PASO 2: EXTRACCIÓN (EXTRACT) DE LOS DATOS
# ===================================================================

# La plataforma datos.gov.co usa la API de Socrata. Podemos usarla para
# descargar los datos directamente, lo que es más eficiente que bajar un CSV.
# Aumentamos el límite para traer más filas (ajusta si es necesario).
api_url = "https://www.datos.gov.co/resource/nudc-7mev.json?$limit=50000"

print(f"📥 Extrayendo datos desde: {api_url}")

try:
    response = requests.get(api_url)
    response.raise_for_status()  # Lanza un error si la petición falla (ej: 404)
    data = response.json()
    df_raw = pd.DataFrame(data)
    print(f"✅ ¡Extracción exitosa! Se cargaron {len(df_raw)} filas.")
    display(df_raw.head())

except requests.exceptions.RequestException as e:
    print(f"❌ Error al extraer los datos: {e}")
    df_raw = pd.DataFrame() # Creamos un dataframe vacío para evitar errores posteriores

except Exception as e:
    print(f"❌ Ocurrió un error inesperado: {e}")
    df_raw = pd.DataFrame()

✅ Librerías importadas.
📥 Extrayendo datos desde: https://www.datos.gov.co/resource/nudc-7mev.json?$limit=50000
✅ ¡Extracción exitosa! Se cargaron 14585 filas.


,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,...,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,2023,05004,Abriaquí,05,Antioquia,3758,Antioquia (ETC),503,62.62,62.62,...,1.96,16.51,2.04,9.52,0,10.46,13.76,2.04,NaN,NaN
1,2023,95025,El Retorno,95,Guaviare,3830,Guaviare (ETC),4438,53.27,53.27,...,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,NaN,NaN
2,2023,95200,Miraflores,95,Guaviare,3830,Guaviare (ETC),2014,32.52,32.52,...,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,NaN,NaN
3,2023,97001,Mitú,97,Vaupés,3831,Vaupés (ETC),10986,59.57,59.57,...,4.04,8.33,4.6,16.18,7.75,21.04,13.84,7.18,NaN,NaN
4,2023,97161,Caruru,97,Vaupés,3831,Vaupés (ETC),1228,51.3,51.3,...,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,NaN,NaN


In [209]:
df_raw

,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,...,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,2023,05004,Abriaquí,05,Antioquia,3758,Antioquia (ETC),503,62.62,62.62,...,1.96,16.51,2.04,9.52,0,10.46,13.76,2.04,NaN,NaN
1,2023,95025,El Retorno,95,Guaviare,3830,Guaviare (ETC),4438,53.27,53.27,...,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,NaN,NaN
2,2023,95200,Miraflores,95,Guaviare,3830,Guaviare (ETC),2014,32.52,32.52,...,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,NaN,NaN
3,2023,97001,Mitú,97,Vaupés,3831,Vaupés (ETC),10986,59.57,59.57,...,4.04,8.33,4.6,16.18,7.75,21.04,13.84,7.18,NaN,NaN
4,2023,97161,Caruru,97,Vaupés,3831,Vaupés (ETC),1228,51.3,51.3,...,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14580,2011,5036,Angelópolis,5,Antioquia,3758,Antioquia (ETC),1707,78.85,78.9,...,3.61,9.5,7.32,0.71,0,0.7,1.08,0,19.57,100
14581,2011,5034,Andes,5,Antioquia,3758,Antioquia (ETC),10244,84.45,84.5,...,0.58,0.04,2.69,5.41,0.73,5.53,6.9,4.11,24.43,93.44
14582,2011,5031,Amalfi,5,Antioquia,3758,Antioquia (ETC),5552,97.71,97.7,...,0,0,0,NaN,0.83,NaN,9.93,4.47,20.01,53.45
14583,2011,5030,Amagá,5,Antioquia,3758,Antioquia (ETC),6631,78.65,78.7,...,6.73,14.46,7.45,0.42,0,0.24,0.91,0,25.05,83.33


## **3. Transformación (Transform)**

Esta es la fase más importante. Aquí limpiamos los datos crudos y los moldeamos para que encajen en nuestro modelo de estrella.

**Pasos:**
1.  **Limpieza y Preparación:** Convertiremos las columnas a los tipos de datos correctos y manejaremos valores faltantes. La columna `matricula` es nuestra métrica principal.
2.  **Creación de Dimensiones:** A partir del DataFrame limpio, crearemos una tabla (DataFrame) para cada dimensión, asegurándonos de que no tengan filas duplicadas y asignando una **llave subrogada** (un ID numérico único).
3.  **Creación de la Tabla de Hechos:** Construiremos la tabla de hechos, que contendrá nuestra métrica (`total_matriculados`) y las llaves foráneas que la conectan a cada dimensión.

## Solución 

1. *Limpieza y Preparación*

In [210]:
df_raw.columns = [col.lower() for col in df_raw.columns]

In [211]:
df_raw.columns = df_raw.columns.str.lower()

In [212]:
print(df_raw.columns.tolist())

['a_o', 'c_digo_municipio', 'municipio', 'c_digo_departamento', 'departamento', 'c_digo_etc', 'etc', 'poblaci_n_5_16', 'tasa_matriculaci_n_5_16', 'cobertura_neta', 'cobertura_neta_transici_n', 'cobertura_neta_primaria', 'cobertura_neta_secundaria', 'cobertura_neta_media', 'cobertura_bruta', 'cobertura_bruta_transici_n', 'cobertura_bruta_primaria', 'cobertura_bruta_secundaria', 'cobertura_bruta_media', 'deserci_n', 'deserci_n_transici_n', 'deserci_n_primaria', 'deserci_n_secundaria', 'deserci_n_media', 'aprobaci_n', 'aprobaci_n_transici_n', 'aprobaci_n_primaria', 'aprobaci_n_secundaria', 'aprobaci_n_media', 'reprobaci_n', 'reprobaci_n_transici_n', 'reprobaci_n_primaria', 'reprobaci_n_secundaria', 'reprobaci_n_media', 'repitencia', 'repitencia_transici_n', 'repitencia_primaria', 'repitencia_secundaria', 'repitencia_media', 'tama_o_promedio_de_grupo', 'sedes_conectadas_a_internet']


In [213]:
df_raw.isnull().sum()

a_o                               0
c_digo_municipio                  0
municipio                         0
c_digo_departamento               0
departamento                      0
c_digo_etc                        0
etc                               0
poblaci_n_5_16                    6
tasa_matriculaci_n_5_16         115
cobertura_neta                  111
cobertura_neta_transici_n        52
cobertura_neta_primaria          91
cobertura_neta_secundaria        94
cobertura_neta_media             93
cobertura_bruta                  68
cobertura_bruta_transici_n       97
cobertura_bruta_primaria         81
cobertura_bruta_secundaria       88
cobertura_bruta_media           127
deserci_n                       142
deserci_n_transici_n            903
deserci_n_primaria              242
deserci_n_secundaria            270
deserci_n_media                 734
aprobaci_n                       25
aprobaci_n_transici_n            93
aprobaci_n_primaria              25
aprobaci_n_secundaria       

In [214]:
print(df_raw.dtypes)

a_o                            object
c_digo_municipio               object
municipio                      object
c_digo_departamento            object
departamento                   object
c_digo_etc                     object
etc                            object
poblaci_n_5_16                 object
tasa_matriculaci_n_5_16        object
cobertura_neta                 object
cobertura_neta_transici_n      object
cobertura_neta_primaria        object
cobertura_neta_secundaria      object
cobertura_neta_media           object
cobertura_bruta                object
cobertura_bruta_transici_n     object
cobertura_bruta_primaria       object
cobertura_bruta_secundaria     object
cobertura_bruta_media          object
deserci_n                      object
deserci_n_transici_n           object
deserci_n_primaria             object
deserci_n_secundaria           object
deserci_n_media                object
aprobaci_n                     object
aprobaci_n_transici_n          object
aprobaci_n_p

In [215]:

columnas_numericas = [
    'poblaci_n_5_16', 'tasa_matriculaci_n_5_16', 'cobertura_neta',
    'cobertura_neta_transici_n', 'cobertura_neta_primaria', 'cobertura_neta_secundaria',
    'cobertura_neta_media', 'cobertura_bruta', 'cobertura_bruta_transici_n',
    'cobertura_bruta_primaria', 'cobertura_bruta_secundaria', 'cobertura_bruta_media',
    'deserci_n', 'deserci_n_transici_n', 'deserci_n_primaria', 'deserci_n_secundaria',
    'deserci_n_media', 'aprobaci_n', 'aprobaci_n_transici_n', 'aprobaci_n_primaria',
    'aprobaci_n_secundaria', 'aprobaci_n_media', 'reprobaci_n', 'reprobaci_n_transici_n',
    'reprobaci_n_primaria', 'reprobaci_n_secundaria', 'reprobaci_n_media',
    'repitencia', 'repitencia_transici_n', 'repitencia_primaria', 'repitencia_secundaria',
    'repitencia_media', 'tama_o_promedio_de_grupo', 'sedes_conectadas_a_internet'
]

# Convertir esas columnas a tipo numérico
for col in columnas_numericas:
    df_raw[col] = pd.to_numeric(df_raw[col], errors='coerce')


In [216]:

#Se relleno los datos nulos con la media para mantener la estructura de la base de datos 
df_raw[columnas_numericas] = df_raw[columnas_numericas].fillna(df_raw[columnas_numericas].mean())


In [217]:
print("Años únicos:", df_raw['a_o'].unique())
print("Departamentos únicos:", df_raw['departamento'].nunique())
print("Municipios únicos:", df_raw['municipio'].nunique())

Años únicos: ['2023' '2022' '2021' '2020' '2019' '2018' '2017' '2016' '2015' '2014'
 '2013' '2012' '2011']
Departamentos únicos: 36
Municipios únicos: 1037


In [218]:

dept_check = df_raw[['c_digo_departamento', 'departamento']].drop_duplicates()
dept_group = dept_check.groupby('c_digo_departamento').agg({'departamento': pd.Series.nunique})
conflictos = dept_group[dept_group['departamento'] > 1]

print(conflictos)

# Error nombre
nombres_conflictivos = df_raw[df_raw['c_digo_departamento'].isin(conflictos.index)][['c_digo_departamento', 'departamento']].drop_duplicates()
print(nombres_conflictivos)

                     departamento
c_digo_departamento              
11                              2
88                              2
     c_digo_departamento                                       departamento
32                    88  Archipiélago de San Andrés, Providencia y Sant...
974                   11                                       Bogotá, D.C.
3389                  88  Archipiélago de San Andrés. Providencia y Sant...
3653                  11                                        Bogotá D.C.


In [219]:
conflictos_municipio = (
    df_raw.groupby('c_digo_municipio')[['municipio', 'c_digo_departamento']]
    .nunique()
    .query('municipio > 1 or c_digo_departamento > 1')
)

print(conflictos_municipio)

                  municipio  c_digo_departamento
c_digo_municipio                                
11001                     2                    1


In [220]:
df_raw['departamento'] = df_raw['departamento'].replace({
    'Bogotá D.C.': 'Bogotá, D.C.',
    'Archipiélago de San Andrés. Providencia y Santa Catalina.': 'Archipiélago de San Andrés, Providencia y Santa Catalina'
})

In [221]:
df_raw.loc[df_raw['c_digo_departamento'] == 88, 'departamento'] = \
    "Archipiélago de San Andrés, Providencia y Santa Catalina"

In [222]:
df_raw['departamento'] = df_raw['departamento'].str.strip().str.replace(r'[.,;]+', '', regex=True)

In [223]:
df_raw['municipio'] = df_raw['municipio'].str.strip().str.replace(r'[.,]', '', regex=True)


2. *Creación de Dimensiones* 

In [224]:
dim_departamento = df_raw[['c_digo_departamento', 'departamento']].drop_duplicates().reset_index(drop=True)
dim_departamento['ct_departamento'] = dim_departamento.index + 1

# Validación
assert dim_departamento['c_digo_departamento'].is_unique, "Hay claves duplicadas en c_digo_departamento"
print("dim_departamento generada correctamente")


dim_departamento generada correctamente


In [225]:
dim_municipio = df_raw[['c_digo_municipio', 'municipio', 'c_digo_departamento']].drop_duplicates().reset_index(drop=True)
dim_municipio['ct_municipio'] = dim_municipio.index + 1

#Validación
assert dim_municipio['c_digo_municipio'].is_unique, "Hay claves duplicadas en c_digo_municipio"
print("dim_municipio generada correctamente")


dim_municipio generada correctamente


In [226]:
dim_tiempo = df_raw[['a_o']].drop_duplicates().reset_index(drop=True)
dim_tiempo['ct_anio'] = dim_tiempo.index + 1 

print("dim_tiempo generada correctamente")


dim_tiempo generada correctamente


3. *Creación de la tabla de Hechos*

In [235]:
df_hechos = df_raw.merge(dim_departamento, on='c_digo_departamento', how='left')
df_hechos = df_hechos.merge(dim_municipio, on='c_digo_municipio', how='left')
df_hechos = df_hechos.merge(dim_tiempo, on='a_o', how='left')



In [228]:
columnas_medidas = [
    'tasa_matriculaci_n_5_16', 'cobertura_neta',
    'reprobaci_n_primaria', 'reprobaci_n_secundaria',
    'reprobaci_n_media', 'repitencia'
]

hechos_final = df_hechos[['ct_departamento', 'ct_municipio', 'ct_anio'] + columnas_medidas]

print(" Tabla de hechos generada")


 Tabla de hechos generada


In [229]:
hechos_final.isnull().sum()

ct_departamento            0
ct_municipio               0
ct_anio                    0
tasa_matriculaci_n_5_16    0
cobertura_neta             0
reprobaci_n_primaria       0
reprobaci_n_secundaria     0
reprobaci_n_media          0
repitencia                 0
dtype: int64

In [231]:
hechos_final.describe()


,ct_departamento,ct_municipio,ct_anio,tasa_matriculaci_n_5_16,cobertura_neta,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia
count,14585.000000,14585.000000,14585.000000,14585.000000,14585.000000,14585.00000,14585.000000,14585.000000,14585.000000
mean,22.047789,581.466027,7.000617,84.971929,85.553196,3.88477,6.809006,4.130812,3.299480
std,8.397619,353.420433,3.741392,18.515676,16.867791,3.78979,6.131056,4.190776,3.353208
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,14.000000,283.000000,4.000000,74.940000,76.950000,0.43000,0.760000,0.680000,0.730000
50%,24.000000,563.000000,7.000000,85.200000,86.270000,3.38000,6.210000,3.160000,2.190000
75%,30.000000,845.000000,10.000000,95.300000,94.470000,5.98000,10.790000,6.360000,5.040000
max,36.000000,1271.000000,13.000000,279.030000,264.540000,51.97000,76.970000,67.860000,37.470000


In [232]:

hechos_final.head()

,ct_departamento,ct_municipio,ct_anio,tasa_matriculaci_n_5_16,cobertura_neta,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia
0,1,1,1,62.62,62.62,1.96,16.51,2.04,9.52
1,2,2,1,53.27,53.27,7.11,9.39,1.75,9.34
2,2,3,1,32.52,32.52,6.93,14.13,7.81,8.65
3,3,4,1,59.57,59.57,4.04,8.33,4.60,16.18
4,3,5,1,51.30,51.30,7.32,15.28,7.27,9.24


In [238]:
hechos_nombres = hechos_final.copy()

hechos_nombres = hechos_nombres.merge(
    dim_municipio[['ct_municipio', 'municipio']],
    on='ct_municipio', how='left'
)

hechos_nombres = hechos_nombres.merge(
    dim_departamento[['ct_departamento', 'departamento']],
    on='ct_departamento', how='left'
)


hechos_nombres = hechos_nombres.merge(
    dim_tiempo[['ct_anio', 'a_o']],  
    on='ct_anio', how='left'
)


In [237]:
hechos_nombres.head()

,ct_departamento,ct_municipio,ct_anio,tasa_matriculaci_n_5_16,cobertura_neta,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,municipio,departamento,a_o
0,1,1,1,62.62,62.62,1.96,16.51,2.04,9.52,Abriaquí,Antioquia,2023
1,2,2,1,53.27,53.27,7.11,9.39,1.75,9.34,El Retorno,Guaviare,2023
2,2,3,1,32.52,32.52,6.93,14.13,7.81,8.65,Miraflores,Guaviare,2023
3,3,4,1,59.57,59.57,4.04,8.33,4.60,16.18,Mitú,Vaupés,2023
4,3,5,1,51.30,51.30,7.32,15.28,7.27,9.24,Caruru,Vaupés,2023


## Preguntas

1. Respecto a la población del municipio ¿Que porcentaje de escolaridad hay?

2. ¿Cómo compararía el rendimiento educativo por municipios?

3. ¿Que departamentos son los que mejor cobertura tienen? ¿Pueden hacer cálculo con SQL?

Este ejercicio se entrega en un archivo Jupyter Notebook (.ipynb) que contenga el código necesario para realizar las consultas en SQL y que previamente haya creado la bodega de datos con un modelo dimensional adecuado.

*Porcentaje de escolaridad por municipio*

In [ ]:

hechos_nombres['tasa_matriculaci_n_5_16'] = pd.to_numeric(hechos_nombres['tasa_matriculaci_n_5_16'], errors='coerce')
hechos_nombres['porcentaje_escolaridad'] = hechos_nombres['tasa_matriculaci_n_5_16'].apply(lambda x: x/10 if x > 100 else x)



In [ ]:
top_escolaridad = hechos_nombres[['departamento', 'municipio', 'porcentaje_escolaridad']].sort_values(by='porcentaje_escolaridad', ascending=False)
print(top_escolaridad)

       departamento                    municipio  porcentaje_escolaridad
14021  Cundinamarca  Villa de San Diego de Ubate                   100.0
8757      Santander           Palmas del Socorro                   100.0
7188      Magdalena                     El Retén                   100.0
7168      Magdalena                Zona Bananera                   100.0
7256          Chocó                     Riosucio                   100.0
...             ...                          ...                     ...
7            Vaupés                     Papunaua                     0.0
2248         Vaupés                     Papunaua                     0.0
1128         Vaupés                     Papunaua                     0.0
5620       Amazonas                  La Victoria                     0.0
6450         Vaupés                     Papunaua                     0.0

[14585 rows x 3 columns]


*Rendiemiento de escolaridad por municipio*

In [247]:
rendimiento = hechos_nombres.groupby('municipio')[[
    'reprobaci_n_primaria', 'reprobaci_n_secundaria', 'reprobaci_n_media', 'repitencia'
]].mean().sort_values(by='repitencia', ascending=False)

print(rendimiento)

               reprobaci_n_primaria  reprobaci_n_secundaria  \
municipio                                                     
Tarapacá                   0.869231                1.861538   
Cumaribo                  17.361136               11.032308   
Puerto Nariño              6.584615                8.795385   
Remolino                   3.428462                4.683077   
Titiribí                   5.206923                8.176154   
...                             ...                     ...   
Charta                     3.971538                8.360769   
Tibirita                   0.513077                0.518462   
Páez                       3.006923                3.859231   
Hacarí                     2.905385                3.414615   
NACIONAL                   0.054200                0.099867   

               reprobaci_n_media  repitencia  
municipio                                     
Tarapacá                5.549231   11.011538  
Cumaribo                6.970832    8.6

*Departamentos con mejor cobertura*

In [248]:
mejor_cobertura = hechos_nombres.groupby('departamento')['cobertura_neta'].mean().sort_values(ascending=False)


In [251]:
!pip install pandasql

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 13.2 MB/s eta 0:00:00
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26892 sha256=fd9bfbe8f4b27cee59a6577e382dfa09bf1ff2e936cc0c5d1d9d02ab0b701110
  Stored in directory: c:\users\usuario\appdata\local\pip\cache\wheels\68\5d\a5\edc271b998f909801d7956959f699b976cc9896075dc47c153
Successfully built pandasql

   ---------------------------------------- 0/3 [greenlet]
   ------------- -------------------------- 1/3 [sqlalchemy]
   ------------- -------------------------- 1/3 [sqlalchemy]
   ------------- -------------------------- 1/3 [sqlalchemy]
   ------------- -------------------------- 1/3 [sqlalchemy]
   ------------- -------------------------- 1/3 [sqlalchemy]
   ------------- -------------------------- 1/3 [sqlalchemy]
   ----------

  DEPRECATION: Building 'pandasql' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pandasql'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [252]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

query = """
SELECT departamento, AVG(cobertura_neta) AS cobertura_promedio
FROM hechos_nombres
GROUP BY departamento
ORDER BY cobertura_promedio DESC
"""

resultado = pysqldf(query)
print(resultado)


                                         departamento  cobertura_promedio
0                                             Quindio           94.579615
1                                               Sucre           93.849172
2                                               Cesar           93.730656
3                                           Magdalena           93.263854
4                                                Meta           90.630439
5                                           Bogotá DC           89.493077
6                                              Tolima           89.239083
7                                        Cundinamarca           88.541257
8                                            Casanare           88.517623
9                                           Antioquia           88.095366
10                                            Córdoba           88.032417
11                                          Atlántico           87.567434
12                                    